In [35]:
# Taken From:
# https://apmonitor.com/wiki/index.php/Main/GekkoPythonOptimization
#
# It was originally created for full PID Control, it has been modified here
# for 'P-only' control of the TCLab device.
#
%matplotlib widget

from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt  

m = GEKKO(remote=False)
#tf = 40
#n = 601 # time points to plot
n = 901 # time points to plot
tf = 600.0 # final time

Ambient_Temp = 23.0

#m.time = np.linspace(0,tf,2*tf+1)
m.time,t_step = np.linspace(0,tf,n, retstep=True)
print('t_step = {0}'.format(t_step))

#step = np.zeros(2*tf+1) + Ambient_Temp
step = np.zeros(len(m.time)) + Ambient_Temp
step[30:]  = 40.0 # If the step is large enough to make the output (OP) saturate (I.e. Want to go over 100%) the model results seem to be inaccurate...

# # Controller model (ORIGINAL)
# Kc = 15.0                    # controller gain
# tauI = 2.0                  # controller reset time
# tauD = 1.0                  # derivative constant

# Controller model
Kc = 4.0                    # controller gain
#tauI = 0.01                  # controller reset time
#tauD = 0.01                  # derivative constant
OP_0  = m.Const(value=0.0)   # OP bias
#OP    = m.Var(value=0.0)       # controller output
#OP    = m.Var(value=0.0, ub=100.0)       # controller output
OP    = m.Var(value=0.0, lb=0.0, ub=100.0)       # controller output
#m.Equation(OP<100.0)
PV    = m.Var(value=Ambient_Temp)       # process variable
SP    = m.Param(value=step)    # set point
Intgl = m.Var(value=0.0)    # integral of the error
err   = m.Intermediate(SP-PV) # set point error
m.Equation(Intgl.dt()==err) # integral of the error

##############################################################
# PID Equation
#                       "P-term"  "I-term"       "D-term???"
#m.Equation(OP == OP_0 + Kc*err + (Kc/tauI)*Intgl - PV.dt())
##############################################################

# P-Only Equation
m.Equation(OP == OP_0 + Kc*err)


P_Term = m.Intermediate(Kc*err)

# Process model (ORIGINAL)
#Kp = 0.5                    # process gain
#tauP = 10.0                 # process time constant

# TCLab FOPDT
Kp = 0.9
tauP = 175.0
thetap = 15.0

# FOPDT Equation
#m.Equation(tauP*PV.dt() + PV == Kp*OP) # ORIGINAL

OP_Delayed=m.Var(value=0)
time_delay_steps = int(thetap / t_step)
print('time_delay_steps = {0}'.format(time_delay_steps))
m.delay(OP,OP_Delayed,time_delay_steps)

#m.Equation(PV.dt()  == (1.0/tauP) * (-(PV - Ambient_Temp)     + Kp*OP))
m.Equation(PV.dt()  == (1.0/tauP) * (-(PV - Ambient_Temp)     + Kp*OP_Delayed))
#          dydt      = (1.0/taup) * (-(y  -  23.0)            + Kp * u)

m.options.SOLVER = 1
m.options.NODES = 2
m.options.COLDSTART = 2
#m.open_folder()

m.options.IMODE=4
m.solve(disp=False)




plt.figure()
plt.subplot(2,2,4)
plt.plot(m.time,OP.value,'b:',label='OP')
plt.ylabel('Output')
plt.grid(True)
plt.ylim(-5,105)
plt.legend()
plt.subplot(2,2,1)
plt.plot(m.time,SP.value,'k-',label='SP')
plt.plot(m.time,PV.value,'r--',label='PV')
plt.grid(True)
plt.ylabel('Process')
plt.legend()

plt.subplot(2,2,2)
plt.plot(m.time,P_Term.value,'g',label='P_Term')
plt.grid()
plt.legend()


plt.subplot(2,2,3)
plt.plot(m.time,err.value,'m--',label='Err')
plt.xlabel('Time (sec)')
plt.grid()
plt.legend()

plt.show()

t_step = 0.6666666666666666
time_delay_steps = 22


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …